# Using Python bindings



Metavision SDK provides Python bindings of the algorithms implemented in C++.

This tutorial presents:

  - How to load and create numpy arrays of events
  - How to do geometrical transformation
  - How to apply noise/ROI filter algorithm
  - How to process events to generate tensor for neural network algorithm

In the tutorial, events are set in a numpy array for the Python bindings and visualized with `matplotlib` as an example.

Let's start with some initial import that we be needed in multiple sections of this tutorial:

In [ ]:
%matplotlib inline

import numpy as np

from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [11, 7]

from metavision_sdk_core import BaseFrameGenerationAlgorithm
import metavision_sdk_ml

def are_events_equal(ev1, ev2):
    """Simple functions comparing event vector field by fields"""
    return ev1.size == ev2.size and min(np.allclose(ev1[name], ev2[name]) for name in ev1.dtype.names)

## Loading events

Before being able to execute algorithms from the SDK, a numpy array of CD events is required.

An empty array of events can be generated as following:

In [ ]:
import metavision_sdk_base
empty_array = np.zeros(2, metavision_sdk_base.EventCD)

print("%r" % empty_array)

In [ ]:
import os
from metavision_core.event_io import RawReader
from metavision_core.utils import get_sample

sequence_filename_raw = "spinner.raw"
# if the file doesn't exist, it will be downloaded from Prophesee's public sample server 
get_sample(sequence_filename_raw, folder=".")

## Geometrical Preprocessing

### Transpose

Swap `x` and `y` coordinates of an event stream.


In [ ]:
import metavision_sdk_cv
transpose = metavision_sdk_cv.TransposeEventsAlgorithm()

Load the events from the RAW files:

In [ ]:
print("before transpose.process(): ")
mv_raw = RawReader(sequence_filename_raw)

ev_0_500 = mv_raw.load_delta_t(500)
print(ev_0_500)

Now transpose the events into a new buffer and check that the event coordinates are transposed


In [ ]:
# Transpose without changing the input array (create a copy)
ev_0_500_transposed = transpose.get_empty_output_buffer()
transpose.process_events(ev_0_500, ev_0_500_transposed)
assert not are_events_equal(ev_0_500_transposed.numpy(), ev_0_500)
print(ev_0_500_transposed.numpy())

Check that the previous buffer is unchanged

In [ ]:
print("after transpose.process(): ")
print(ev_0_500)  # unchanged

To avoid data copy and allocation, the function `process_events_()` can be used to process the events inside the provided buffer:

In [ ]:
# Transpose in-place
transpose.process_events_(ev_0_500)
assert are_events_equal(ev_0_500_transposed.numpy(), ev_0_500)
print("After transpose.process_events_(): ")
print(ev_0_500)

### FlipX / FlipY

FlipX applies a vertical line symmetry in the middle of the image to transform all events coordinates.
Whereas FlipY applies a horizontal one.
 
Before flipping the coordinates, let's load some events from the file.


In [ ]:
mv_raw = RawReader(sequence_filename_raw)

ev = mv_raw.load_delta_t(600)
print(ev)

Instantiate FlipX algorithm and apply it:

In [ ]:
import metavision_sdk_core
sensor_width = mv_raw.get_size()[1]
print("sensor width: %s every x coordinate should be now x - sensor_width - 1" % sensor_width)
flipX = metavision_sdk_core.FlipXAlgorithm(sensor_width - 1)
ev_flipX_buffer = flipX.get_empty_output_buffer()
flipX.process_events(ev, ev_flipX_buffer)
print(ev_flipX_buffer.numpy())

Instantiate FlipY algorithm and apply it:

In [ ]:
sensor_height = mv_raw.get_size()[0]

print("sensor height: %s every y coordinate should be now y - sensor_height - 1" % sensor_height)

flipY = metavision_sdk_core.FlipYAlgorithm(sensor_height - 1)
ev_flipY_buffer = flipY.get_empty_output_buffer()
flipY.process_events(ev, ev_flipY_buffer)
print(ev_flipY_buffer.numpy())

As for the transpose algorithm, the buffer can be processed in place with the function `process_events_()`

In [ ]:
flipX.process_events_(ev)
flipY.process_events_(ev)
print(ev)

## Event filtering

### Region Of Interest (ROI)

The Region Of Interest Algorithm filters out all events that are outside of a rectangular region of interest.
The filter takes two coordinates as arguments: one coordinate per rectangle corner (the top left and the right bottom corners).

In the following code, the RoiFilter is instantiated to remove all events outside of a box represented by its corners coordinates:

  - top left: (200, 100);
  - bottom right: (320, 200).

In [ ]:
roi = metavision_sdk_core.RoiFilterAlgorithm(x0=100, y0=255, x1=200, y1=320)
ev_filtered_buffer = roi.get_empty_output_buffer()

mv_raw = RawReader(sequence_filename_raw)

ev = mv_raw.load_delta_t(1000000)

roi.process_events(ev, ev_filtered_buffer)
ev_filtered = ev_filtered_buffer.numpy()
print(ev_filtered)

To show that all the events are in the provided ROI is easier to check with an image:

In [ ]:
height, width = mv_raw.get_size()
frame = np.zeros((height, width, 3), dtype=np.uint8)
BaseFrameGenerationAlgorithm.generate_frame(ev_filtered, frame)
image = plt.imshow(frame[..., ::-1])

### Noise Filters: Trail / STC

Trail and STC filters are used to reduce the number of events produced by the camera.

First, load some events.

In [ ]:
mv_raw = RawReader(sequence_filename_raw)
height, width = mv_raw.get_size()
mv_raw.seek_time(2e6)

ev = mv_raw.load_delta_t(10000)

Instantiate the noise filtering algorithms and apply them on the buffer of events

In [ ]:
trail = metavision_sdk_cv.TrailFilterAlgorithm(width=width, height=height, threshold=10000)
ev_trail_buf = trail.get_empty_output_buffer()
trail.process_events(ev, ev_trail_buf)
ev_trail_np = ev_trail_buf.numpy()

stc = metavision_sdk_cv.SpatioTemporalContrastAlgorithm(width=width, height=height, threshold=10000)
ev_stc_buf = stc.get_empty_output_buffer()
stc.process_events(ev, ev_stc_buf)
ev_stc_np = ev_stc_buf.numpy()

To compare the behavior of the algorithms, the generated frames are displayed side by side with the number of events

In [ ]:
plt.rcParams['figure.figsize'] = [18, 7]
_, (ax1, ax2, ax3) = plt.subplots(1, 3)
ax1.set_title("unfiltered : {} events".format(len(ev)))
BaseFrameGenerationAlgorithm.generate_frame(ev, frame)
ax1.imshow(frame[..., ::-1])
ax2.set_title("filtered with trail : {} events".format(len(ev_trail_np)))
BaseFrameGenerationAlgorithm.generate_frame(ev_trail_np, frame)
ax2.imshow(frame[..., ::-1])
ax3.set_title("filtered with STC : {} events".format(len(ev_stc_np)))
BaseFrameGenerationAlgorithm.generate_frame(ev_stc_np, frame)
image = ax3.imshow(frame[..., ::-1])

## Event preprocessing : CDProcessing



In [ ]:
# the visualization functions from metavision_ml can be used.
from metavision_ml.preprocessing import viz_histo

We create a `CDProcessing` function with:

  * `histo`
  * delta_t = 50ms
  * network input size as a half of the size of the event_frame

In [ ]:
mv_raw = RawReader(sequence_filename_raw)
height, width = mv_raw.get_size()
mv_raw.seek_time(3e6)

ev = mv_raw.load_delta_t(50000)

cdproc = metavision_sdk_ml.CDProcessing.create_CDProcessingHisto(delta_t=50000, 
                             network_input_width=width // 2,
                             network_input_height=height // 2,
                             event_input_width=width, event_input_height=height)

frame_buffer = cdproc.init_output_tensor()
cdproc.process_events(3000000, ev, frame_buffer)
print("frame_buffer shape        : ", frame_buffer.shape)
print("number of non-zero values : ", np.sum(frame_buffer != 0))
print("set of unique values      :\n", np.around(np.unique(frame_buffer), decimals=2))
plt.imshow(viz_histo(frame_buffer))

In [ ]:
ev = mv_raw.load_delta_t(50000)

# reset and reuse the frame_buffer
frame_buffer.fill(0)
cdproc.process_events(3050000, ev, frame_buffer)
print("frame_buffer shape        : ", frame_buffer.shape)
print("number of non-zero values : ", np.sum(frame_buffer != 0))
print("set of unique values      :\n", np.around(np.unique(frame_buffer), decimals=2))
plt.imshow(viz_histo(frame_buffer))